In [2]:
import pandas as pd
import numpy as np
import re

In [57]:
data_path = '/data/ephemeral/home/data/df_high_ratio.csv'
df = pd.read_csv(data_path)

In [58]:
# 영어가 있는 행만 추출하여 확인
english_rows = df[df['text'].str.contains(r'[A-Za-z]', regex=True)]

print(english_rows['text'])

0                  사건!>실"를 b$#라 #극)]체O.;월f:?
1             전W;참c 이nd 유j]"m객ie((우려` ?<. 개조
2             더*} ]i대+ 김현권!v원a}_대8*vdL줄!알았d 
3             $G ;!p 서울 |곳 나vo객 =적"bs 귀s,d거)
4             nO통령z\후 p싱턴으로 ^R!與지p부 @re-#{{\
                        ...                 
1587    G창 k천K 진입도로 .방도 승격 국P 50V억 지원 4차로 8장
1589                     특징주 메가B디(자회사 합병에 급W
1590                   ]광공사 페루 여0자J무장T도 R의해Y
1592                   프랑스 이란에 핵OW 의무 %0L삼가라
1593                  최저임W위 퇴장 |견하는 한국I총 위원B
Name: text, Length: 1580, dtype: object


In [59]:
# 두 글자 이상 연속된 영어 알파벳이 포함된 행 필터링
english_two_or_more = df[df['text'].str.contains(r'[A-Za-z]{2,}', regex=True)]

print(english_two_or_more['text'])

1            전W;참c 이nd 유j]"m객ie((우려` ?<. 개조
2            더*} ]i대+ 김현권!v원a}_대8*vdL줄!알았d 
3            $G ;!p 서울 |곳 나vo객 =적"bs 귀s,d거)
4            nO통령z\후 p싱턴으로 ^R!與지p부 @re-#{{\
5          ^r^[P홀w2019년H'v요?서트vYp 세,#uNs[cJ
                       ...                 
1579       UEFA 챔스리그 4강 마드리드 더비 성사 레알 vs 아 
1582       프로농구 kt 로건 없이도 삼성 완파 벌써 지난해 10승 
1584                      KT Be Y 폰 2 단독 출시
1586    확진 속출F부산 사하구B모든 학교 이틀간 RV수업3q7"교 대3
1592                  프랑스 이란에 핵OW 의무 %0L삼가라
Name: text, Length: 1065, dtype: object


In [60]:
# 두 글자 이상 연속된 소문자 알파벳이 포함된 행 필터링
english_two_or_more = df[df['text'].str.contains(r'[a-z]{2,}', regex=True)]

print(english_two_or_more['text'][70:79])

117       R*_jz )침 공]기준'나왔다Pu구자 ]정부담k
119               y=sc럼*광합성하h f공장W 개발
121    MWC 스키점프nVev `d한PI글라데r1'|x아~나\
122    선두5도X공사*GS칼텍$^K;dhb현Q#i탈7m연승]합
123    미국 7R; 멕시g 2hoU년m\o컵X공"=W8<청#듯
124    밀라노!d지오rd4 광장k`시a j/|I노Q8o옥외광고
125    노?u김)d 피QY3손민j 겨4P{c는bd베G트n연$곡
126    .달 CES %굴#N바@은^새a|더o폰I중저o폰O rb
127    롯데\cs론트pKB]리din 승o두AON드블럼은e5연}
Name: text, dtype: object


In [61]:
# 대문자 두 글자 이상의 단어나 'vs'를 제외하고 영어를 삭제하는 함수
def remove_unwanted_english(text):
    # 대문자 두 글자 이상 또는 'vs'는 유지, 나머지 영어 단어는 제거
    return re.sub(r'(?![A-Z]{2,}|vs)[A-Za-z]', '', text)

# 예제 문장
text = "MWC 스키점프nVev `d한PI글라데r1'|x아~나"

# 함수 적용
cleaned_text = remove_unwanted_english(text)
print(cleaned_text)


MW 스키점프 `한P글라데1'|아~나


In [66]:
def remove_noise(text):
    # 대문자 두 글자 이상의 단어 또는 'vs'를 찾는 정규 표현식
    meaningful_pattern = r'[A-Z]{2,}|vs'
    
    # 결과를 담을 리스트
    cleaned_text = []

    current_word = ''
    for char in text:
        if re.match(r'[\uAC00-\uD7A3]', char):  # 한글인 경우
            if current_word:
                # 이전에 쌓인 단어가 있으면 의미 있는지 확인
                if re.match(meaningful_pattern, current_word):
                    cleaned_text.append(current_word)
                current_word = ''  # current_word 초기화
            cleaned_text.append(char)  # 한글 추가
        elif char.isalpha():  # 영어 알파벳인 경우
            current_word += char  # 영어 문자를 current_word에 추가
        else:  # 기타 문자는 그대로 유지
            if current_word:
                if re.match(meaningful_pattern, current_word):
                    cleaned_text.append(current_word)
                current_word = ''  # current_word 초기화
            cleaned_text.append(char)  # 기타 문자는 추가

    # 마지막으로 쌓인 current_word가 의미 있는지 확인
    if current_word and re.match(meaningful_pattern, current_word):
        cleaned_text.append(current_word)

    return ''.join(cleaned_text)

# 예시 텍스트
text = "전W참c이nd유jm객ie우vs려개조VS이건예시TEXT입니다.TET"
cleaned_text = remove_noise(text)
print(cleaned_text)

df['cleaned_text']= df['text'].apply(remove_noise)

전참이유객우vs려개조VS이건예시TEXT입니다.TET


In [56]:
from transformers import BertTokenizer, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
tokens = tokenizer.tokenize(cleaned_text)

print(tokens)

['전', '##참', '##이', '##유', '##객', '##우', '##v', '##s', '##려', '##개', '##조', '##V', '##S', '##이', '##건', '##예', '##시', '##TE', '##X', '##T', '##입니다', '.', 'TE', '##T']


In [ ]:
df['cleaned_text']

0        사건!>실"를 $#라 #극)]체.;월:?
1        전;참 이 유]"객((우려` ?<. 개조
2       더*} ]대+ 김현권!원}_대8*줄!알았 
3        $ ;! 서울 |곳 나객 =적" 귀,거)
4         통령\후 싱턴으로 ^!지부 @-#{{\
                 ...           
1590          ]광공사 페루 여0자무장도 의해
1591      대한항공 인천∼바르셀로나 노선 첫 취항
1592       프랑스 이란에 핵OW 의무 %0삼가라
1593        최저임위 퇴장 |견하는 한국총 위원
1594    옥션 5만∼10만원 한우 설 선물세트 인기
Name: cleaned_text, Length: 1595, dtype: object

## [MASK] 토큰으로 노이즈 정상화

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# BERT 모델과 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
model = BertForMaskedLM.from_pretrained('klue/bert-base')

# 입력 텍스트 예시
text = "롯데[MASK]론트[MASK]리[MASK] 승[MASK]두[MASK]드블럼은[MASK]연."

# 텍스트 토큰화 및 입력 ID 생성
input_ids = tokenizer.encode(text, return_tensors='pt')
mask_token_indices = torch.where(input_ids == tokenizer.mask_token_id)[1]

# [MASK] 위치 예측
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs.logits


# 각 [MASK] 위치에 대해 가장 높은 확률의 예측 단어 ID를 가져와서 토큰화 해제
for mask_index in mask_token_indices:
    predicted_token_id = predictions[0, mask_index, :].argmax(axis=-1)
    input_ids[0, mask_index] = predicted_token_id

# 최종 문장 디코딩
final_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
print("BERT가 채운 문장:", final_text)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT가 채운 문장: 롯데 자이언츠 론트. 리 승 승. 두 번 드블럼은 한 연.
